# Datastory!

In [2]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd

from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
#jskfjklsd



In [3]:
import pandas as pd
sea_df = pd.read_csv('Global_sea_level_rise.csv')
dist_df = pd.read_csv('1900_2021_DISASTERS.xlsx - emdat data.csv')

tsunami_df = dist_df[dist_df['Disaster Subtype'].str.lower() == 'tsunami'].copy()
flood_df = dist_df[dist_df['Disaster Subtype'].str.lower() == 'flood'].copy()

tsunami_df.columns = tsunami_df.columns.str.strip()
flood_df.columns = flood_df.columns.str.strip()
sea_df.rename(columns={'year': 'Year'}, inplace=True)

#subsets om samen te voegen
tsunami_subset = tsunami_df[['Year', 'Disaster Subtype', 'Country', 'Region', 'Continent', 'Origin', 'River Basin',
                             'Total Deaths', 'Total Affected', "Total Damages ('000 US$)", 'Latitude', 'Longitude', 'Location']]

flood_subset = flood_df[['Year', 'Disaster Subtype', 'Country', 'Region', 'Continent', 'Origin', 'River Basin',
                         'Total Deaths', 'Total Affected', "Total Damages ('000 US$)", 'Latitude', 'Longitude', 'Location']]

#eerste twee samen voegen
disaster_df = pd.concat([tsunami_subset, flood_subset], ignore_index=True)

#laatste samenvoegen
merged_df = disaster_df.merge(sea_df[['Year', 'mmfrom1993-2008average']], on='Year', how='left')

#missende waardes invullen met 0
merged_df = merged_df.fillna(0)




Hieronder worden de argumenten voor het eerste perspectief uitgwerkt.

## Tsunami’s vormen een toenemend gevaar in de toekomst omdat naar mate de zeespiegel stijgt overstromingen na een tsunami toenemen.
In het figuur is te zien als de tijd toeneemt, neemt de zeespiegel ook toe. Maar dan nemen de effecten van de overstromingen ook toe, wat resulteert in meer doden en meer schade.

In [ ]:
import numpy as np 
merged_df['Total Deaths'] = pd.to_numeric(merged_df['Total Deaths'], errors='coerce').fillna(0)
merged_df['Total Affected'] = pd.to_numeric(merged_df['Total Affected'], errors='coerce').fillna(0)
merged_df["Total Damages ('000 US$)"] = pd.to_numeric(merged_df["Total Damages ('000 US$)"], errors='coerce').fillna(0)

merged_df['Impact_Score'] = merged_df[['Total Deaths', 'Total Affected', "Total Damages ('000 US$)"]].max(axis=1)

merged_df = merged_df[(merged_df['Latitude'] != 0) & (merged_df['Longitude'] != 0)]

filtered_df = merged_df[merged_df['Impact_Score'] > 0].copy()

filtered_df['Impact_Score_Log'] = filtered_df['Impact_Score'].apply(lambda x: np.log10(x + 1))

fig = px.scatter_geo(
    filtered_df,
     lat='Latitude',
    lon='Longitude',
    color='mmfrom1993-2008average',
    size='Impact_Score_Log',
    size_max=40,
    hover_name='Country',
    hover_data={
        'Year': True,
        'Impact_Score': True,
        'Total Deaths': True,
        'Total Affected': True,
        "Total Damages ('000 US$)": True,
        'mmfrom1993-2008average': ':.2f'
    },
    animation_frame='Year',
    projection='natural earth',
    color_continuous_scale='Viridis',
    title='Overstromingen en tsunami\'s met zeespiegelstijging en impact (log schaal)'
)

fig.update_layout(height=700)

fig.show()

## Tsunami’s vormen een toenemend gevaar in de toekomst omdat naar mate de zeespiegel stijgt kosten van damages en total deaths toenemen na een tsunami.

Onze verwachting van dit argument was eigenlijk dat naar mate de zeespiegel stijgt de kosten van schade ook toegenomen zouden zijn. Wat we helaas niet verwachtte is dat de dataset hierover beperkte informaite bevatte. Er zijn drie verschillende plots gemaakt (die op elkaar een vallen) die de zeespiegelstijging, tsunami's en kosten van de schade van deze tsunami's visueel weergeven. Er zijn slechts 2 datapunten in de dataset in deze kolom, waarschijnlijk omdat landen dit niet hebben genoteerd of omdat de informatie niet bereikbaar was. Wat wel opvallend is om te zien dat de totaal aantal doden een enorme stijging heeft in 2004. Dit is het jaar waar een tsunami op 26 december 2004 de kust van Thailand trof, wat resulteerde in meer dan tweehonderd duizend doden. Ook is te zien als er naar aantal doden per jaar wordt gekeken dat wel toeneemt. Dit kan komen doordat de zeespiegelstijging ervoor zorgt dat er meer volume is waarmee een tsunami schade kan doen. Hieronder is dit te zien.

In [ ]:
import plotly.graph_objects as go

merged_df['Year'] = merged_df['Year'].astype(int)
df = merged_df[merged_df['Region'].str.contains('Asia|Oceania', case=False, na=False)]

yearly_deaths = df.groupby('Year')['Total Deaths'].sum()
yearly_damages = df.groupby('Year')["Total Damages ('000 US$)"].sum()
yearly_sealevel = df.groupby('Year')['mmfrom1993-2008average'].mean()

# Pas alleen Total Damages van 2011 aan:
yearly_damages.loc[2011] = 10_800_000  # 10.8 miljoen

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=yearly_deaths.index,
    y=yearly_deaths.values,
    mode='lines+markers',
    name='Total Deaths',
    line=dict(color='red')
))

fig.add_trace(go.Scatter(
    x=yearly_damages.index,
    y=yearly_damages.values,
    mode='lines+markers',
    name="Total Damages ('000 US$)",
    line=dict(color='orange')
))

fig.add_trace(go.Scatter(
    x=yearly_sealevel.index,
    y=yearly_sealevel.values,
    mode='lines+markers',
    name='Sea Level Rise (mm)',
    line=dict(color='royalblue')
))

fig.update_layout(
    title="Tsunamis in Azië en Oceanië: Totale Doden, Schade en Zeespiegelstijging per Jaar (2011 aangepast)",
    xaxis_title="Year",
    yaxis_title="Value",
    height=600
)

fig.show()


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Zorg dat merged_df al geladen is en 'Year' int is
merged_df['Year'] = merged_df['Year'].astype(int)

# Filter regio Azië en Oceanië via Region
df = merged_df[merged_df['Region'].str.contains('Asia|Oceania', case=False, na=False)]

years = sorted(df['Year'].unique())

# Berekeningen per jaar
sea_level_avg = df.groupby('Year')['mmfrom1993-2008average'].mean().reset_index()
deaths_per_year = df.groupby('Year')['Total Deaths'].sum().reset_index()
damages_per_year = df.groupby('Year')["Total Damages ('000 US$)"].sum().reset_index()

# Maak subplots: 3 rijen, 1 kolom, alle lijnen
fig = make_subplots(
    rows=3, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=("Total Damages ('000 US$)", "Total Deaths", "Sea Level Rise (mm)")
)

# Voeg lijn diagram voor Total Damages
fig.add_trace(
    go.Scatter(
        x=damages_per_year['Year'],
        y=damages_per_year["Total Damages ('000 US$)"],
        mode='lines+markers',
        name='Total Damages',
        line=dict(color='orange')
    ),
    row=1, col=1
)

# Voeg lijn diagram voor Total Deaths
fig.add_trace(
    go.Scatter(
        x=deaths_per_year['Year'],
        y=deaths_per_year['Total Deaths'],
        mode='lines+markers',
        name='Total Deaths',
        line=dict(color='red')
    ),
    row=2, col=1
)

# Voeg lijn diagram voor Sea Level Rise
fig.add_trace(
    go.Scatter(
        x=sea_level_avg['Year'],
        y=sea_level_avg['mmfrom1993-2008average'],
        mode='lines+markers',
        name='Sea Level Rise',
        line=dict(color='royalblue')
    ),
    row=3, col=1
)

fig.update_layout(
    height=800,
    title_text="Tsunamis in Azië en Oceanië: Schade, Doden en Zeespiegelstijging per Jaar",
    showlegend=False
)

fig.update_xaxes(title_text="Year", row=3, col=1)
fig.update_yaxes(title_text="Damages ('000 US$)", row=1, col=1)
fig.update_yaxes(title_text="Total Deaths", row=2, col=1)
fig.update_yaxes(title_text="Sea Level Rise (mm)", row=3, col=1)

fig.show()


In [ ]:
df = merged_df[merged_df['Region'].str.contains('Asia|Oceania', case=False, na=False)].copy()

# Zet NaN damages op 0 zodat ze ook getoond worden, maar kleiner
df["Total Damages ('000 US$)"] = df["Total Damages ('000 US$)"].fillna(0)

# Eventueel filteren op geldige lat/lon
df = df.dropna(subset=['Latitude', 'Longitude'])

fig = px.scatter_geo(
    df,
    lat='Latitude',
    lon='Longitude',
    hover_name='Location',
    hover_data={
        'Year': True,
        "Total Damages ('000 US$)": ':.2f',
        'Total Deaths': True
    },
    size="Total Damages ('000 US$)",
    size_max=30,
    color="Total Damages ('000 US$)",
    color_continuous_scale='OrRd',
    projection='natural earth',
    title="Locaties van Tsunamis in Azië en Oceanië met Total Damages"
)

fig.update_geos(
    showcountries=True,
    countrycolor="LightGray",
    showcoastlines=True,
    coastlinecolor="Gray",
    lataxis_range=[-50, 50],
    lonaxis_range=[60, 180],
)

fig.update_layout(height=600, margin={"r":0,"t":40,"l":0,"b":0})

fig.show()


Nu de argumenten voor het tweede perspectief

##  De hoeveelheid van tsunami’s is niet/minder evenredig toegenomen naar mate de zeespiegel stijgt. 
Hieronder wordt een grafiek gemaakt die het verband tussen de zeespiegel en het aantal tsunami's laat zien. 

In [ ]:
import matplotlib.pyplot as plt

import pandas as pd

disasters = pd.read_csv('1900_2021_DISASTERS.xlsx - emdat data.csv')
sea_level = pd.read_csv('Global_sea_level_rise.csv')

tsunamis = disasters[disasters['Disaster Subtype'].isin(['Tsunami', 'Tsunami/Tidal wave'])]


tsunami_counts = tsunamis.groupby('Year').size().reset_index(name='Tsunami Count')

combined = pd.merge(tsunami_counts, sea_level, left_on='Year', right_on='year', how='inner')
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.scatter(combined['mmfrom1993-2008average'], combined['Tsunami Count'], alpha=0.6, color='darkblue')
plt.title('Zeespiegel vs. aantal tsunami’s per jaar')
plt.xlabel('Zeespiegel (mm t.o.v. 1993–2008 gemiddelde)')
plt.ylabel('Aantal tsunami’s')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
correlation = combined['Tsunami Count'].corr(combined['mmfrom1993-2008average'])
(f'Correlatiecoëfficiënt: {correlation:.2f}')

Er is een erg zwakke correlatie ontdekt (r = 0.17) tussen de zeespiegelstijging en het aantal tsunami’s per jaar. Dit betekent dat de stijgende zeespiegel niet direct samenhangt met een toename van het aantal tsunami’s, en dus vooral bepaald blijft door geologische factoren.


## De zeespiegelstijging versterkt het effect van de tsunami maar veroorzaakt het niet.
Hieronder worden twee grafieken gemaakt, de eerste laat de zeespiegel vs. aantal doden door tsunami's zien, en de tweede laat de zeespiegel vs. het aantal tsunami's per jaar zien. Vervolgens wordt hieruit geanalyseerd of de stijgende zeespiegel het effect van de tsunami verorzaakt of versterkt. 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

disasters = pd.read_csv('1900_2021_DISASTERS.xlsx - emdat data.csv')
sea_level = pd.read_csv('Global_sea_level_rise.csv')

tsunamis = disasters[disasters['Disaster Subtype'].isin(['Tsunami', 'Tsunami/Tidal wave'])]

tsunami_deaths = tsunamis.groupby('Year')['Total Deaths'].sum().reset_index()

combined = pd.merge(tsunami_deaths, sea_level, left_on='Year', right_on='year', how='inner')

plt.figure(figsize=(10, 5))
plt.scatter(combined['mmfrom1993-2008average'], combined['Total Deaths'], alpha=0.7, color='firebrick')

plt.title('Zeespiegel vs. aantal doden door tsunami’s')
plt.xlabel('Zeespiegel (mm t.o.v. 1993–2008 gemiddelde)')
plt.ylabel('Totaal aantal doden per jaar')
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

disasters = pd.read_csv('1900_2021_DISASTERS.xlsx - emdat data.csv')
sea_level = pd.read_csv('Global_sea_level_rise.csv')

tsunamis = disasters[disasters['Disaster Subtype'].isin(['Tsunami', 'Tsunami/Tidal wave'])]

tsunami_counts = tsunamis.groupby('Year').size().reset_index(name='Aantal Tsunamis')

combined_counts = pd.merge(tsunami_counts, sea_level, left_on='Year', right_on='year', how='inner')

plt.figure(figsize=(10, 5))
plt.scatter(combined_counts['mmfrom1993-2008average'], combined_counts['Aantal Tsunamis'], alpha=0.6, color='darkblue')
plt.title('Zeespiegel vs. Aantal tsunami’s per jaar')
plt.xlabel('Zeespiegel (mm t.o.v. 1993–2008 gemiddelde)')
plt.ylabel('Aantal tsunami’s')
plt.grid(True)
plt.tight_layout()
plt.show()


De grafieken tonen aan dat de zeespiegelstijging niet leidt tot meer tsunami’s, maar wel dat het effect ervan verergert. Bij een hogere zeespiegel zie je namelijk meer doden, maar niet meer tsunami's relatief gezien.

##  Tsunami’s ontstaan vooral in specifieke aardbevingzones - niet in regios met hoogste zeespiegelstijging.


In [ ]:
import pandas as pd
import pandas as pd

disasters = pd.read_csv('1900_2021_DISASTERS.xlsx - emdat data.csv')
sea_level = pd.read_csv('Global_sea_level_rise.csv')

tsunamis = disasters[disasters['Disaster Subtype'].isin(['Tsunami', 'Tsunami/Tidal wave'])]

tsunami_deaths = tsunamis.groupby('Year')['Total Deaths'].sum().reset_index()
combined = pd.merge(tsunami_deaths, sea_level, left_on='Year', right_on='year', how='inner')

plt.figure(figsize=(10, 5))
plt.scatter(combined['mmfrom1993-2008average'], combined['Total Deaths'], alpha=0.6, color='firebrick')
plt.title('Zeespiegel vs. aantal doden door tsunami’s')
plt.xlabel('Zeespiegel (mm t.o.v. 1993–2008 gemiddelde)')
plt.ylabel('Totaal aantal doden')
plt.grid(True)
plt.tight_layout()
plt.show()

De grafiek geeft weer hoe erg het aantal doden door tsunamis en de ontwikkeling ervan over jaren zich verhoudt tot de stijging van de zeespiegel. zelfs bij hele hoge waarden is een bijna stabiele hoeveelheid telkens te zien van het aantal doden met de toename van de zeespiegel.

## Bewijs 3)Tsunami’s ontstaan vooral in specifieke aardbevingzones - niet in regios met hoogste zeespiegelstijging.

In [ ]:
tsunami_counts = tsunamis.groupby('Year').size().reset_index(name='Tsunami Count')

print(sea_level.columns)

combined = pd.merge(tsunami_counts, sea_level, left_on='Year', right_on='year', how='inner')

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.scatter(combined['mmfrom1993-2008average'], combined['Tsunami Count'], alpha=0.6, color='darkblue')
plt.title('Zeespiegel vs. aantal tsunami’s per jaar')
plt.xlabel('Zeespiegel (mm t.o.v. 1993–2008 gemiddelde)')
plt.ylabel('Aantal tsunami’s')
plt.grid(True)
plt.tight_layout()
plt.show()

De grafiek hierboven laat het aantal tsunamis met de zeespiegel zien er is te zien dat de ontwikkeling van de hoeveelheid tsunamis niet drastisch verandert om een oorzakelijk verband te kunnen vinden eventueel.